In [2]:
import numpy as np
import nltk
import os
import ast

In [3]:
cwd = os.getcwd()
print(cwd)
corpusDir = os.path.join(cwd, 'data/lightweight')
print(corpusDir)

/data/Workspace/DeepChat
/data/Workspace/DeepChat/data/lightweight


In [4]:
lines = {}
with open(os.path.join(corpusDir, 'movie_lines.txt'), 'r', encoding='iso-8859-1') as f:
    for line in f:
        #print(line)
        fields = line.split(' +++$+++ ')
        #print(fields)
        obj = {}
        obj['lineID'] = fields[0]
        obj['characterID'] = fields[1]
        obj['movieID'] = fields[2]
        obj['characterName'] = fields[3]
        obj['text'] = fields[4]
        lines[fields[0]] = obj
#print(lines)

In [5]:
conversations = []
with open(os.path.join(corpusDir, 'movie_conversations.txt'), 'r', encoding='iso-8859-1') as f:
    for line in f:
        #print(line)
        fields = line.split(' +++$+++ ')
        #print(fields)
        obj = {}
        obj['character1ID'] = fields[0]
        obj['character2ID'] = fields[1]
        obj['movieID'] = fields[2]
        #obj['lineIDs'] = fields[3]
        #print(obj)
        lineIDs = ast.literal_eval(fields[3])
        #print(lineIDs)
        obj['lineIDs'] = lineIDs
        #print(obj)
        obj['lines'] = []
        for lineID in lineIDs:
            #print(lineID, "--", lines[lineID])
            obj['lines'].append(lines[lineID])
        conversations.append(obj)
#print(conversations)

In [7]:
wordIDMap = {}
IDWordMap = {}
unknownToken = -1

In [19]:
def getWordID(word, shouldAddToDict=True):
    word = word.lower()
    wordID = wordIDMap.get(word, -1)
    if wordID == -1:
        if shouldAddToDict:
            wordID = len(wordIDMap)
            wordIDMap[word] = wordID
            IDWordMap[wordID] = word
        else:
            wordID = unknownToken
    return wordID

In [36]:
for conversation in conversations:
    #print(conversation)
    for i in range(len(conversation['lines']) - 1):
        #print(conversation['lines'][i])
        inputStatement = conversation['lines'][i]
        #print(inputStatement)
        replyStatement = conversation['lines'][i + 1]
        inputWords = getWordsFromLine(inputStatement['text'])
        replyWords = getWordsFromLine(replyStatement['text'], True)
        #print(inputWords)
        #print(replyWords)

In [34]:
sentMaxLength = 10 #maximum length of an input or output sentence
def getWordsFromLine(line, isReply=False):
    '''Returns the word IDs from the vovabulary'''
    words = []
    sentences = nltk.sent_tokenize(line)
    #print(sentences)
    # Since we are limited by a maxmimum length of sentences, we keep the last lines if the statement is a question/input
    # and the first few lines if the statement is an answer/reply
    for i in range(len(sentences)):
        if not isReply:
            i = len(sentences) - 1 - i
        tokensFromCurrSent = nltk.word_tokenize(sentences[i])
        #print(tokensFromCurrSent)
        if len(words) + len(tokensFromCurrSent) > sentMaxLength:
            break
        else:
            temp = []
            for token in tokensFromCurrSent:
                temp.append(getWordID(token))
            if isReply:
                words = words + temp
            else:
                words = temp + words # Append in the reverse order because we're considering the last few lines
    return words